In [ ]:
import populator

In [ ]:
data_dir = 'data'
# Second item in tuple is max batch size for requests.
exchanges = [('binance', 1000)]
tick_size = '1h'
start = '2000-01-01T00:00:00Z'

In [ ]:
stables = ['TUSD/USDT', 'PAX/USDT', 'USDC/USDT', 'USDC/TUSD', 'USDS/TUSD', 'USDC/PAX', 'USDS/USDT']
populator.populate(data_dir, exchanges, stables, tick_size, start)
frames = [(pair, populator.load_data_as_frame(data_dir, 'binance', pair, tick_size, start)) for pair in stables]

In [ ]:
import backtrader as bt

In [ ]:
starting_cash = 10**5
class StableCoinStrategy(bt.Strategy):
    def next(self):
        if not self.position:
            # Backtrader craps out for some reason with round numbers...
            self.buy(size=starting_cash / 2 + 1)
        elif self.data.close[0] > 1.0 and self.position.size >= 1000:
            self.sell(size=1000)
        elif self.data.close[0] < 1.0 and self.broker.getcash() * self.data.close[0] >= 1000:
            self.buy(size=1000)

In [ ]:
def run_strategy(name, df):
    print('Running strategy for {}.'.format(name))
    cerebro = bt.Cerebro()
    cerebro.addstrategy(StableCoinStrategy)
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.00, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    cerebro.broker.set_coc(True)
    cerebro.broker.setcash(starting_cash)
    cerebro.broker.setcommission(commission=0.001)
    sim = cerebro.run()
    
    print('Final Value: {}'.format(cerebro.broker.getvalue()))
    print('PNL %: {}'.format(100.0 * (cerebro.broker.getvalue() / starting_cash) - 100.0))
    print('Sharpe Ratio: {}'.format(sim[0].analyzers.sharpe.get_analysis()['sharperatio']))
    print('Max Drawdown: {}'.format(sim[0].analyzers.drawdown.get_analysis().max.drawdown))
    print()

In [ ]:
for (name, df) in frames:
    run_strategy(name, df)